# Cuantización

## Conceptos Clave

Según la Clase 06, la cuantización es el proceso de convertir una amplitud de señal continua a un número finito de niveles discretos. Cuando una señal muestreada $x(n)$ se cuantiza, se obtiene una señal cuantizada $x_Q(n)$, y la diferencia entre ellas es el **error de cuantización** $e_q(n) = x_Q(n) - x(n)$.

Si asumimos que el error de cuantización está uniformemente distribuido entre $-\Delta/2$ y $\Delta/2$, donde $\Delta$ es el tamaño del escalón de cuantización, el valor cuadrático medio (RMS) del error de cuantización es:
$$e_{rms} = \sqrt{E[e_q^2]} = \frac{\Delta}{\sqrt{12}}$$

La relación entre el rango de la señal $R$ (la diferencia entre el valor máximo y mínimo de la señal) y el número de bits $B$ se da por el número de niveles $2^B$:
$$R = (2^B - 1) \Delta \approx 2^B \Delta$$De donde podemos despejar el tamaño del escalón:$$\Delta = \frac{R}{2^B}$$

El **Signal-to-Quantization Noise Ratio (SQNR)** es una medida de la calidad de la señal cuantizada, definida como la relación entre la potencia de la señal y la potencia del ruido de cuantización, a menudo expresada en decibelios (dB):
$$SQNR_{dB} = 10 \log_{10} \left( \frac{P_x}{P_{e_q}} \right)$$Para una señal sinusoidal de amplitud pico $A$, con $P_x = A^2/2$ y $e_{rms}^2 = \Delta^2/12$, y considerando $R = 2A$, el SQNR teórico aproximado en función del número de bits es:$$SQNR_{dB} \approx 6.02 B + 1.76$$
Una regla de dedo común es que cada bit adicional en la cuantización mejora el SQNR en aproximadamente 6 dB.

## Ejemplo 1: Cálculo de Bits para una Señal con Rango y Error Específicos

Este ejemplo se basa en el cálculo presentado en la Clase 06 para determinar el número de bits de cuantización necesarios para una señal con un rango dinámico dado y un error RMS de cuantización máximo permitido.

**Problema:** Se tiene una señal con un rango dinámico de $R = 10$ (por ejemplo, de -5V a +5V). Se desea que el error RMS de cuantización sea menor o igual a $e_{rms} = 50 \mu V = 50 \times 10^{-6} V$. ¿Cuántos bits de cuantización $B$ se necesitan? ¿Cuál es el SQNR teórico para este número de bits?

Usamos las fórmulas:
1.  $e_{rms} = \frac{\Delta}{\sqrt{12}} \implies \Delta = e_{rms} \sqrt{12}$
2.  $\Delta = \frac{R}{2^B} \implies 2^B = \frac{R}{\Delta} = \frac{R}{e_{rms} \sqrt{12}}$
3.  $B = \log_2 \left( \frac{R}{e_{rms} \sqrt{12}} \right)$
4.  $SQNR_{dB} \approx 6.02 B + 1.76$

### Código Python para Ejemplo 1

In [1]:
# --------------- Imports ---------------
import numpy as np
import math

# --- Ejemplo 1: Cálculo de Bits y SQNR ---
print("--- Ejemplo 1: Cálculo de Bits y SQNR ---")

# Parámetros dados
R = 10.0  # Rango de la señal
erms_desired = 50e-6 # Error RMS de cuantización deseado (50 microVolts)

print(f"Rango de la señal (R): {R} V")
print(f"Error RMS deseado (erms_deseado): {erms_desired*1e6:.2f} uV")

# Calcular Delta basado en el error RMS deseado
Delta_calculated = erms_desired * math.sqrt(12)
print(f"Tamaño del escalón de cuantización requerido (Delta): {Delta_calculated:.6f} V")

# Calcular el número de niveles (2^B)
levels_float = R / Delta_calculated
print(f"Número de niveles (aproximado 2^B): {levels_float:.2f}")

# Calcular el número de bits (B)
# Usamos ceil para asegurar que el error sea MENOR o igual al deseado
B_calculated = math.ceil(math.log2(levels_float))

print(f"Número de bits de cuantización requeridos (B): {B_calculated}")

# Calcular el error RMS real con el número de bits calculado
Delta_actual = R / (2**B_calculated)
erms_actual = Delta_actual / math.sqrt(12)
print(f"Tamaño del escalón real con {B_calculated} bits (Delta_real): {Delta_actual:.6f} V")
print(f"Error RMS real con {B_calculated} bits (erms_real): {erms_actual*1e6:.2f} uV")

# Calcular el SQNR teórico (aproximado 6B + 1.76)
sqnr_theoretical_db = 6.02 * B_calculated + 1.76
print(f"SQNR teórico con {B_calculated} bits: {sqnr_theoretical_db:.2f} dB")

print("\n--- Fin del Ejemplo 1 ---\n")

--- Ejemplo 1: Cálculo de Bits y SQNR ---
Rango de la señal (R): 10.0 V
Error RMS deseado (erms_deseado): 50.00 uV
Tamaño del escalón de cuantización requerido (Delta): 0.000173 V
Número de niveles (aproximado 2^B): 57735.03
Número de bits de cuantización requeridos (B): 16
Tamaño del escalón real con 16 bits (Delta_real): 0.000153 V
Error RMS real con 16 bits (erms_real): 44.05 uV
SQNR teórico con 16 bits: 98.08 dB

--- Fin del Ejemplo 1 ---



## Ejemplo Extra: Visualización Interactiva del Efecto de la Cuantización

Este ejemplo te permite explorar cómo la cuantización afecta una señal al cambiar el número de bits. Podrás ver la señal original, la señal cuantizada y el error de cuantización, observando cómo la forma del error y el SQNR cambian con el número de bits.

La cuantización se realiza dividiendo el rango de la señal en $2^B$ niveles uniformes y asignando cada muestra de la señal continua al nivel más cercano.

In [ ]:
# --------------- Imports ---------------
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import IntSlider, VBox, interactive_output, Layout
from IPython.display import display, clear_output
import math

# --- Generar una señal de ejemplo ---
fs = 100 # Frecuencia de muestreo (arbitraria para la demo)
t = np.arange(0, 1, 1/fs) # Vector de tiempo
# Una señal compuesta por dos sinusoides con amplitudes y frecuencias diferentes
# y un poco de ruido para hacerla más realista antes de cuantizar
signal_analog = 0.7 * np.sin(2 * np.pi * 5 * t) + 0.3 * np.sin(2 * np.pi * 15 * t + np.pi/4) + 0.05 * np.random.randn(len(t))

# Rango de la señal (max - min)
signal_range = np.max(signal_analog) - np.min(signal_analog)
signal_min = np.min(signal_analog)

# --- Función de Cuantización ---
def quantize_signal(signal, B, signal_range, signal_min):
    """Cuantiza una señal a B bits."""
    if B < 1:
        # Si B es menor a 1, esencialmente no hay niveles de cuantización significativos
        # Podemos retornar una señal cuantizada a 0 (o al valor medio del rango)
        # El error será la señal original (desplazada si el medio no es 0)
        # El SQNR será muy bajo o indefinido (representado por -inf o un valor muy bajo)
        mean_val = signal_min + signal_range/2
        error = signal - mean_val # Error relativo a la "cuantización" a un solo nivel
        power_signal = np.mean((signal - np.mean(signal))**2) # Potencia AC de la señal
        power_error = np.mean(error**2)
        sqnr_db = 10 * np.log10(power_signal / power_error) if power_error > 0 else -np.inf
        return np.full_like(signal, mean_val), error, sqnr_db


    num_levels = 2**B
    Delta = signal_range / num_levels

    # Calcular los bordes de los escalones
    # Los niveles están centrados, los bordes están a Delta/2 de distancia
    # El primer borde está en signal_min
    quantization_edges = signal_min + Delta * np.arange(num_levels + 1)

    # Asignar cada muestra al índice del escalón
    # signal_analog >= edge[i] and signal_analog < edge[i+1]
    # np.digitize retorna el índice del bin *derecho* al que pertenece cada elemento.
    # Los bins son definidos por quantization_edges.
    quantized_indices = np.digitize(signal, quantization_edges) - 1
    # Clamp para asegurar que los índices estén dentro del rango [0, num_levels - 1]
    quantized_indices = np.clip(quantized_indices, 0, num_levels - 1)


    # Calcular el valor cuantizado como el centro del escalón correspondiente
    signal_quantized = signal_min + quantized_indices * Delta + Delta/2

    # Calcular el error de cuantización
    quantization_error = signal_quantized - signal

    # Calcular el SQNR numérico (en dB)
    # Calcular la potencia de la señal (componente AC)
    power_signal = np.mean((signal - np.mean(signal))**2)
    # Calcular la potencia del error
    power_error = np.mean(quantization_error**2)

    # Evitar log10 de cero o negativo
    if power_error <= 1e-15: # Considerar error muy pequeño como cero
        sqnr_db = np.inf
    elif power_signal <= 1e-15: # Si la señal no tiene potencia (ej. DC), SQNR no es significativo
         sqnr_db = -np.inf # O algún otro indicador, -inf sugiere mucho ruido comparado con la señal
    else:
        sqnr_db = 10 * np.log10(power_signal / power_error)


    return signal_quantized, quantization_error, sqnr_db

# --- Función de Actualización para el Widget Interactivo ---
def update_quantization_plot(B):
    """Actualiza los gráficos al cambiar el número de bits."""
    # Limpiar la salida anterior
    with output_area:
        clear_output(wait=True)

    signal_quantized, quantization_error, sqnr_db = quantize_signal(
        signal_analog, B, signal_range, signal_min
    )

    # --- Creación de Subplots ---
    fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    fig.suptitle(f'Efecto de la Cuantización (B = {B} bits)', fontsize=14)

    # Plot 1: Señal Original y Cuantizada
    axes[0].plot(t, signal_analog, label='Original', alpha=0.7)
    axes[0].plot(t, signal_quantized, label='Cuantizada', alpha=0.9)
    axes[0].set_ylabel('Amplitud')
    axes[0].set_title('Señal Original vs. Cuantizada')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_ylim(signal_min - signal_range*0.1, signal_min + signal_range*1.1) # Fija límites Y para comparación

    # Plot 2: Error de Cuantización
    axes[1].plot(t, quantization_error, color='red', alpha=0.7)
    axes[1].set_ylabel('Error')
    axes[1].set_title('Error de Cuantización')
    axes[1].grid(True)
    # Ajustar límite Y basado en Delta para B >= 1
    if B >= 1:
        error_limit = signal_range / (2**B) / 2 # Error max es Delta/2
        axes[1].set_ylim(-error_limit * 2, error_limit * 2) # Ajustar límite Y
    else: # Para B < 1, el error es ~la señal original
         axes[1].set_ylim(-signal_range*0.6, signal_range*0.6) # Rango más amplio

    # Plot 3: SQNR
    # Mostramos el SQNR en el título
    sqnr_text = f'SQNR Numérico: {sqnr_db:.2f} dB' if sqnr_db != np.inf and sqnr_db != -np.inf else ('SQNR Numérico: Infinito dB (Error cero)' if sqnr_db == np.inf else 'SQNR Numérico: Indefinido o Muy Bajo dB')
    # Graficar una línea horizontal que muestre el valor de SQNR (si es finito)
    if sqnr_db != np.inf and sqnr_db != -np.inf:
        axes[2].axhline(sqnr_db, color='green', linestyle='--', label=f'SQNR = {sqnr_db:.2f} dB')
        axes[2].set_ylim(max(-10, sqnr_db - 10), sqnr_db + 10) # Ajustar límite Y alrededor del SQNR
        axes[2].legend()
    else:
         axes[2].text(t[len(t)//2], 0, sqnr_text, ha='center', va='center', color='green', fontsize=12) # Mostrar texto si SQNR es inf/indefinido
         axes[2].set_ylim(-10, 70) # Rango fijo si SQNR no es finito

    axes[2].set_ylabel('SQNR (dB)')
    axes[2].set_title(f'SQNR en dB') # Título más general
    axes[2].set_xlabel('Tiempo (s)')
    axes[2].grid(True)


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar layout para el título principal

    # Mostrar el gráfico dentro del contexto del output_area
    with output_area:
        plt.show()

# --- Widget de Control ---
style = {'description_width': 'initial'}
layout_slider = Layout(width='80%')

B_slider = IntSlider(
    min=0, max=10, step=1, value=3, # Rango de bits de 0 a 10 (0 para ver caso extremo)
    description='Número de Bits (B):',
    style=style,
    layout=layout_slider
)

# --- Contenedor de Salida ---
output_area = widgets.Output()

# --- Conectar Widget y Mostrar ---
interactive_plot = interactive_output(
    update_quantization_plot,
    {'B': B_slider}
)

print("Ajusta el slider para cambiar el número de bits de cuantización:")

display(VBox([B_slider, output_area]))

# Ejecutar la función inicialmente para mostrar el estado por defecto
with output_area:
    update_quantization_plot(B_slider.value)

print("\n--- Fin del Código Interactivo ---\n")

Ajusta el slider para cambiar el número de bits de cuantización:



--- Fin del Código Interactivo ---

